# Modelo Poisson para predicción de partidos de futbol

El modelo poisson es una herramienta de la probabilidad que tiene multiples uso en la medición de vida util de un evento, no obstante, esta distribucion también es usada para teimpos de espera en problemas de procesos estocasticos, en este proyecto se utilizará la distribución Poisson como herramienta de medición de calculo para obtener la probabilidad de que un equipo anote $X$ determinada cantidad de goles. 

$$ f_x(X) = \frac{e^{-\lambda}\lambda^{x}}{x!} = P[X = x] $$

In [1]:
#bibliotecas a utilizar
import numpy as np
import pandas as pd
import math

In [2]:
data = pd.read_excel('Info_clean_2.xlsx')
data.head(5)

,idPartido,temporada,division,jornada,EquipoLocal,EquipoVisitante,Fecha3,Hora,Goles_L,Goles_V,Unnamed: 10
0,1,2007_1_C,1,1,Estudiantes Tecos,U.A.N.L.,2007-01-19,16:55:00,2,1,NaN
1,2,2007_1_C,1,1,Querétaro,Veracruz,2007-01-20,15:00:00,0,0,NaN
2,3,2007_1_C,1,1,Necaxa,Chiapas,2007-01-20,16:55:00,3,1,NaN
3,4,2007_1_C,1,1,Morelia,Atlante,2007-01-20,16:55:00,3,1,NaN
4,5,2007_1_C,1,1,Monterrey,Santos Laguna,2007-01-20,16:55:00,1,0,NaN


In [3]:
#revision de nulos
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3519 entries, 0 to 3518
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   idPartido        3519 non-null   int64         
 1   temporada        3519 non-null   object        
 2   division         3519 non-null   int64         
 3   jornada          3519 non-null   int64         
 4   EquipoLocal      3519 non-null   object        
 5   EquipoVisitante  3519 non-null   object        
 6   Fecha3           3519 non-null   datetime64[ns]
 7   Hora             3519 non-null   object        
 8   Goles_L          3519 non-null   int64         
 9   Goles_V          3519 non-null   int64         
 10  Unnamed: 10      0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(5), object(4)
memory usage: 302.5+ KB


In [4]:
#borrando columna llena de nulos 
del data['Unnamed: 10']

In [5]:
# check efectivo de que si se haya borrado la columna llena de nulos
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3519 entries, 0 to 3518
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   idPartido        3519 non-null   int64         
 1   temporada        3519 non-null   object        
 2   division         3519 non-null   int64         
 3   jornada          3519 non-null   int64         
 4   EquipoLocal      3519 non-null   object        
 5   EquipoVisitante  3519 non-null   object        
 6   Fecha3           3519 non-null   datetime64[ns]
 7   Hora             3519 non-null   object        
 8   Goles_L          3519 non-null   int64         
 9   Goles_V          3519 non-null   int64         
dtypes: datetime64[ns](1), int64(5), object(4)
memory usage: 275.0+ KB


In [6]:
#definicion de columnas con las cuales trabajar
columns_work = ['idPartido' ,'temporada' ,'jornada' ,'EquipoLocal' ,'EquipoVisitante' ,'Goles_L' ,'Goles_V']
soccer = data.copy()
soccer = soccer[columns_work]

In [7]:
soccer.head(2)

,idPartido,temporada,jornada,EquipoLocal,EquipoVisitante,Goles_L,Goles_V
0,1,2007_1_C,1,Estudiantes Tecos,U.A.N.L.,2,1
1,2,2007_1_C,1,Querétaro,Veracruz,0,0


Nota: la modelación de este proyecto esta basada en que se tomara un torneo anterior para modelar todo el torneo
en tiempo presente. Esto se decidio despues de un analsisis y serie de pruebas con otros modelos 

In [8]:
# filtrado, en la temporada (t-1)
mask = (soccer['temporada'] == '2017_2_A')
soccer = soccer[mask]

In [9]:
# particion de la data y configuracion por local y visitante
locales = soccer.copy() 
del locales['EquipoVisitante']
locales = locales.rename(columns = {'EquipoLocal':'equipo',
                                    'Goles_L':'goles_favor',
                                    'Goles_V':'goles_contra'
                                    })
visitantes = soccer.copy() 
del visitantes['EquipoLocal']
visitantes = visitantes.rename(columns = {'EquipoVisitante':'equipo',
                                    'Goles_L':'goles_contra',
                                    'Goles_V':'goles_favor'
                                    })

In [10]:
#promedio general de goles anotados como local y visitante
prom_gol_l = soccer['Goles_L'].mean() 
prom_gol_v = soccer['Goles_V'].mean()

In [11]:
prom_gol_v, prom_gol_l

(1.1503267973856208, 1.392156862745098)

In [12]:
#promedio de goles por equipo del torneo pasado LOCALES
soccer_l_prom = locales.groupby(['equipo']).agg({'goles_favor':'mean' ,'goles_contra':'mean'})
soccer_l_prom = soccer_l_prom.rename(columns = {'goles_favor':'prom_gf_l',
                                    'goles_contra':'prom_gc_l',
                                    })
#VISITANTES
soccer_v_prom = visitantes.groupby(['equipo']).agg({'goles_favor':'mean' ,'goles_contra':'mean'})
soccer_v_prom = soccer_v_prom.rename(columns = {'goles_favor':'prom_gf_v',
                                    'goles_contra':'prom_gc_v',
                                    })

Factor de ataque: una especie de pondracion para los goles anotados, es decir, es un promedio ponderado de los goles que un equipo memte como local/visitante entre el promedio general 

In [13]:
#LOCALES
#factor de ataque
soccer_l_prom['f_a_l'] = soccer_l_prom['prom_gf_l'] / prom_gol_l
#factor de defensa 
soccer_l_prom['f_d_l'] = soccer_l_prom['prom_gc_l'] / prom_gol_v

In [14]:
#VISITANTES
#factor de ataque
soccer_v_prom['f_a_v'] = soccer_v_prom['prom_gf_v'] / prom_gol_v
#factor de defensa 
soccer_v_prom['f_d_v'] = soccer_v_prom['prom_gc_v'] / prom_gol_l

ABT (analytical Base Table): tabla base para poder aplicar el modelo matematico que resolverá el promedio

In [15]:
abt = soccer_l_prom.merge(soccer_v_prom, on = ['equipo'])
abt.head(2)

,prom_gf_l,prom_gc_l,f_a_l,f_d_l,prom_gf_v,prom_gc_v,f_a_v,f_d_v
equipo,,,,,,,,
Atlas,1.000000,0.75,0.718310,0.651989,1.666667,1.444444,1.448864,1.037559
Club América,1.285714,1.00,0.923541,0.869318,1.400000,1.100000,1.217045,0.790141


los torneos que se buscan predecir son aquellos que son los complementarios a un año calnedario futbol, es decir, el torneo en tiempo $t$

In [16]:
#los resultados que buscamos predecir son los del torneo 'actual'
# por eso ahora tomaremos un data set con el torneo complementario del anio
soccer_to_pred = data.copy()
soccer_to_pred = soccer_to_pred[columns_work]
mask_pred      = (soccer_to_pred['temporada'] == '2018_1_C')
soccer_to_pred = soccer_to_pred[mask_pred]


In [17]:
soccer_to_pred.head(3)

,idPartido,temporada,jornada,EquipoLocal,EquipoVisitante,Goles_L,Goles_V
3366,3367,2018_1_C,1,Puebla,U.A.N.L.,2,1
3367,3368,2018_1_C,1,Atlas,León,1,2
3368,3369,2018_1_C,1,Cruz Azul,Tijuana,0,0


In [18]:
#unimos las tablas de la condicion de visitante y local para tener 
# todos los factores que implican al modelado

merge_1 =  soccer_to_pred.merge(abt, left_on = 'EquipoLocal', right_on = 'equipo')

In [19]:
merge_1.head(3)

,idPartido,temporada,jornada,EquipoLocal,EquipoVisitante,Goles_L,Goles_V,prom_gf_l,prom_gc_l,f_a_l,f_d_l,prom_gf_v,prom_gc_v,f_a_v,f_d_v
0,3367,2018_1_C,1,Puebla,U.A.N.L.,2,1,1.111111,0.888889,0.798122,0.772727,0.5,1.5,0.434659,1.077465
1,3385,2018_1_C,3,Puebla,Veracruz,2,0,1.111111,0.888889,0.798122,0.772727,0.5,1.5,0.434659,1.077465
2,3403,2018_1_C,5,Puebla,Guadalajara,2,0,1.111111,0.888889,0.798122,0.772727,0.5,1.5,0.434659,1.077465


In [20]:
columns_pred_work = ['idPartido', 'temporada',
		 'jornada', 'EquipoLocal', 'EquipoVisitante',
		 'Goles_L', 'Goles_V', 'f_a_l', 'f_d_l'] #,'f_a_v', 'f_d_v'

In [21]:
merge_1 = merge_1[columns_pred_work]

In [22]:
# uniendo la data de los visitantes
merge_2 = merge_1.merge(abt, left_on = 'EquipoVisitante', right_on = 'equipo')

In [23]:
merge_2.head(2)

,idPartido,temporada,jornada,EquipoLocal,EquipoVisitante,Goles_L,Goles_V,f_a_l_x,f_d_l_x,prom_gf_l,prom_gc_l,f_a_l_y,f_d_l_y,prom_gf_v,prom_gc_v,f_a_v,f_d_v
0,3367,2018_1_C,1,Puebla,U.A.N.L.,2,1,0.798122,0.772727,2.125,0.375,1.526408,0.325994,1.222222,1.444444,1.0625,1.037559
1,3508,2018_1_C,16,Necaxa,U.A.N.L.,1,1,0.957746,0.772727,2.125,0.375,1.526408,0.325994,1.222222,1.444444,1.0625,1.037559


In [24]:
columns_pred_work = ['idPartido', 'temporada',
		 'jornada', 'EquipoLocal', 'EquipoVisitante',
		 'Goles_L', 'Goles_V', 'f_a_l_x', 'f_d_l_x', 'f_a_v', 'f_d_v']

In [25]:
merge_2 = merge_2[columns_pred_work]

Calculo de las lambdas para los parametros de la poisson: en este proyecto el parametro lambda es el promedio ponderado de los goles por equipo

In [26]:
lambdas = merge_2.copy()

# el factor de ataque local * factor de defensa de la visita * promedio de goles formaran la lambda
lambdas['lambda_l'] = lambdas['f_a_l_x'] * lambdas['f_d_v'] * prom_gol_l
lambdas['lambda_v'] = lambdas['f_a_v'] * lambdas['f_d_l_x'] * prom_gol_v

In [27]:
#maximo y minimo de goles del torneo anterior
# para establecer como rango de la poisson, es decir,
# por donde debe de correr la x
max_goals = soccer['Goles_L'].max()
min_goals = soccer['Goles_L'].min()

In [28]:
min_goals, max_goals

(0, 6)

In [29]:
def pois_goals_p(lambd:float ,num_goals:int) -> float:
    ''' Esta función calcula la probabilidad de meter 
    num_goals en un partido con el parametro lambda dado
    
    >>> pois_goals_p(3, 4)
    0.16803135574154082

    esto se puede traducir como:
    la probabilidad de meter 4 goles dado que el 
    promedio de goles del equipo es de 3
    '''
    return ( ( (math.exp(-lambd)) * (lambd ** num_goals) ) / (math.factorial(num_goals)) )

In [30]:
#probabilidades de meter n goles para el equipo local
for i in range(min_goals ,max_goals + 1):
    lambdas[f'P_GL({i})'] = lambdas['lambda_l'].apply(pois_goals_p, num_goals = i)


In [31]:
#probabilidades de meter n goles para el equipo visitante
for i in range(min_goals ,max_goals + 1):
    lambdas[f'P_GV({i})'] = lambdas['lambda_v'].apply(pois_goals_p, num_goals = i)

In [32]:
pre_final = lambdas.copy()
pre_final = pre_final[['idPartido', 'temporada', 'jornada', 'EquipoLocal', 'EquipoVisitante','Goles_L', 'Goles_V', 
                        'P_GL(0)', 'P_GL(1)', 'P_GL(2)', 'P_GL(3)', 'P_GL(4)', 'P_GL(5)', 'P_GL(6)', 
                        'P_GV(0)', 'P_GV(1)', 'P_GV(2)', 'P_GV(3)', 'P_GV(4)', 'P_GV(5)', 'P_GV(6)']]

In [33]:
pre_final.head(2)

,idPartido,temporada,jornada,EquipoLocal,EquipoVisitante,Goles_L,Goles_V,P_GL(0),P_GL(1),P_GL(2),...,P_GL(4),P_GL(5),P_GL(6),P_GV(0),P_GV(1),P_GV(2),P_GV(3),P_GV(4),P_GV(5),P_GV(6)
0,3367,2018_1_C,1,Puebla,U.A.N.L.,2,1,0.315738,0.363996,0.209815,...,0.023238,0.005358,0.001029,0.388896,0.36729,0.173443,0.054602,0.012892,0.002435,0.000383
1,3508,2018_1_C,16,Necaxa,U.A.N.L.,1,1,0.250722,0.346851,0.239919,...,0.038264,0.010587,0.002441,0.388896,0.36729,0.173443,0.054602,0.012892,0.002435,0.000383


In [34]:
# inicializamos nuevas variables para establecer la proba de que gane el local
# visitante o haya empate
pre_final['P_Local'] = 0
pre_final['P_Visit'] = 0
pre_final['P_Empat'] = 0

In [35]:
pre_final.head(2)

,idPartido,temporada,jornada,EquipoLocal,EquipoVisitante,Goles_L,Goles_V,P_GL(0),P_GL(1),P_GL(2),...,P_GV(0),P_GV(1),P_GV(2),P_GV(3),P_GV(4),P_GV(5),P_GV(6),P_Local,P_Visit,P_Empat
0,3367,2018_1_C,1,Puebla,U.A.N.L.,2,1,0.315738,0.363996,0.209815,...,0.388896,0.36729,0.173443,0.054602,0.012892,0.002435,0.000383,0,0,0
1,3508,2018_1_C,16,Necaxa,U.A.N.L.,1,1,0.250722,0.346851,0.239919,...,0.388896,0.36729,0.173443,0.054602,0.012892,0.002435,0.000383,0,0,0


El siguiente ´while´ es para multiplicar todas las probas marginales de que un equipo gane, es decir, la proba de que el local gane es: Todos aquellos resultados en los cuales el local anota más goles que el visitante, es por es que, se tiene que multiplicar
$$ P[Local] = \sum_{i<j}^n P[Goles_Local(i)] > P[Goles_Visitante(j)] $$

In [36]:

while max_goals > min_goals:
    pre_final['P_Local'] = pre_final['P_Local'] + (pre_final[f'P_GL({max_goals})'] * pre_final[f'P_GV({max_goals - 1})'])
    pre_final['P_Visit'] = pre_final['P_Visit'] + (pre_final[f'P_GV({max_goals})'] * pre_final[f'P_GL({max_goals - 1})'])
    pre_final['P_Empat'] = pre_final['P_Empat'] + (pre_final[f'P_GL({max_goals})'] * pre_final[f'P_GV({max_goals})'])

    max_goals -= 1

In [37]:
pre_final.head(2)

,idPartido,temporada,jornada,EquipoLocal,EquipoVisitante,Goles_L,Goles_V,P_GL(0),P_GL(1),P_GL(2),...,P_GV(0),P_GV(1),P_GV(2),P_GV(3),P_GV(4),P_GV(5),P_GV(6),P_Local,P_Visit,P_Empat
0,3367,2018_1_C,1,Puebla,U.A.N.L.,2,1,0.315738,0.363996,0.209815,...,0.388896,0.36729,0.173443,0.054602,0.012892,0.002435,0.000383,0.233944,0.191654,0.174799
1,3508,2018_1_C,16,Necaxa,U.A.N.L.,1,1,0.250722,0.346851,0.239919,...,0.388896,0.36729,0.173443,0.054602,0.012892,0.002435,0.000383,0.244430,0.166870,0.175568


In [38]:
final = pre_final.copy()

In [39]:
final = final[['temporada', 'jornada', 'EquipoLocal', 'EquipoVisitante',
       'Goles_L', 'Goles_V', 'P_Local', 'P_Visit', 'P_Empat']]

In [40]:
final['preds'] = 'SA'

In [41]:
final.head(2)

,temporada,jornada,EquipoLocal,EquipoVisitante,Goles_L,Goles_V,P_Local,P_Visit,P_Empat,preds
0,2018_1_C,1,Puebla,U.A.N.L.,2,1,0.233944,0.191654,0.174799,SA
1,2018_1_C,16,Necaxa,U.A.N.L.,1,1,0.244430,0.166870,0.175568,SA


In [42]:
# asignacion de variable gana local
mask_local = (final['P_Local'] > final['P_Visit'])
final.loc[mask_local, 'preds'] = 'L'

# asignacion de variable gana visitante
mask_visit = (final['P_Local'] < final['P_Visit'])
final.loc[mask_visit, 'preds'] = 'V'

# asignacion de variable empate
mask_empt = (( (final['P_Empat'] > final['P_Local']) & (final['P_Empat'] > final['P_Visit']) ) 
            | ((abs(final['P_Local'] - final['P_Visit'])) < 0.05) )
final.loc[mask_empt, 'preds'] = 'E'

In [43]:
# etiqueta de resultados reales
final['result'] = 'SA'

res_local = ( final['Goles_L'] > final['Goles_V'] )
res_visit = ( final['Goles_L'] < final['Goles_V'] )
res_empat = ( final['Goles_L'] == final['Goles_V'] )

final.loc[res_local, 'result'] = 'L'
final.loc[res_visit, 'result'] = 'V'
final.loc[res_empat, 'result'] = 'E'

In [44]:
final['preds'].value_counts(), final['result'].value_counts() 

(L    66
 E    49
 V    38
 Name: preds, dtype: int64,
 L    67
 E    45
 V    41
 Name: result, dtype: int64)

El modelo al menos en proporciones las respeta, pero nos interesa saber, cuantas predicciones correctas se hicieron

In [45]:
final.head(2)

,temporada,jornada,EquipoLocal,EquipoVisitante,Goles_L,Goles_V,P_Local,P_Visit,P_Empat,preds,result
0,2018_1_C,1,Puebla,U.A.N.L.,2,1,0.233944,0.191654,0.174799,E,L
1,2018_1_C,16,Necaxa,U.A.N.L.,1,1,0.244430,0.166870,0.175568,L,E


In [46]:
acuracy = final['preds'] == final['result']

In [47]:
x = list(acuracy.value_counts())

precision = x[1] / sum(x)
print(f'La precision del modelo es de {round(precision*100,2)} %')

La precision del modelo es de 40.52 %


Elaborado por: Ramirez Montes Jonathan Natael